# Segment 2

## Regression but Not Backwards 

In [1]:
# loading libraries
import warnings
warnings.simplefilter("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_score
from ggplot import * #needed?
from sklearn.preprocessing import LabelBinarizer

In [2]:
# define column names
names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species']

# loading training data
iris = pd.read_csv('iris.data', header=None, names=names)

In [3]:
enc = LabelBinarizer().fit_transform(iris['species']) # one hot encode categorical species feature
X_reg = np.concatenate([iris[['sepal_length', "sepal_width", "petal_width"]],enc[:,:2]], axis=1) # end index is exclusive
y_reg = np.array(iris['petal_length']) # column name is another way of indexing df

# split into train and test
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X_reg, y_reg, test_size=0.33, random_state=42)

### Linear Regression

In [4]:
from sklearn import linear_model

In [5]:
import statsmodels.api as sm

from IPython.core.display import HTML

def short_summary(est):
    return HTML(est.summary().tables[1].as_html())

#### Using Sklearn

Train Model

In [6]:
# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(X_train_reg, y_train_reg)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [7]:
# The coefficients
print('Sklearn Coefficients: \n', regr.coef_)
print('Sklearn Intercept: \n', regr.intercept_)

Sklearn Coefficients: 
 [ 0.61545727 -0.12598231  0.63387466 -1.92974128 -0.47748   ]
Sklearn Intercept: 
 0.6100972575054717


Test Model

In [8]:
# Make predictions using the testing set
y_pred = regr.predict(X_test_reg)
print("Sklearn MSE: ", mean_squared_error(y_test_reg, y_pred))

Sklearn MSE:  0.0733764940898916


#### Using Statsmodels

Train Model

In [9]:
X_train_regSM = sm.add_constant(X_train_reg)

# Fit and summarize OLS model
mod = sm.OLS(y_train_reg, X_train_regSM)
res = mod.fit()

In [10]:
print("Statmodels Coefficients:")
short_summary(res)

Statmodels Coefficients:


,coef,std err,t,P>|t|,[0.025,0.975]
const,0.6101,0.364,1.677,0.097,-0.112,1.332
x1,0.6155,0.058,10.620,0.000,0.500,0.731
x2,-0.1260,0.106,-1.187,0.238,-0.337,0.085
x3,0.6339,0.146,4.347,0.000,0.344,0.923
x4,-1.9297,0.303,-6.376,0.000,-2.531,-1.329
x5,-0.4775,0.115,-4.168,0.000,-0.705,-0.250


Test Model

In [11]:
X_test_regSM = sm.add_constant(X_test_reg)
y_pred = res.predict(X_test_regSM) 
print("Statsmodels MSE: ", mean_squared_error(y_test_reg, y_pred))

Statsmodels MSE:  0.07337649408989215


In [12]:
#add plots? normality etc, fitted line 3D

### Multivariate Regression

In [13]:
X_MVreg = np.concatenate([iris[['sepal_length']], enc[:,:2]], axis=1) # , "sepal_width" missing
y_MVreg = np.array(iris[['petal_length', 'petal_width']]) # 2 response vars

# split into train and test
X_train_MVreg, X_test_MVreg, y_train_MVreg, y_test_MVreg = train_test_split(X_MVreg, y_MVreg, test_size=0.33, random_state=42)

print("Regression X_train:") 
print(X_train_MVreg[1:5,])
print("\nRegression y_train:") 
print(y_train_MVreg[1:5])

Regression X_train:
[[7.6 0.  0. ]
 [5.6 0.  1. ]
 [5.1 1.  0. ]
 [7.7 0.  0. ]]

Regression y_train:
[[6.6 2.1]
 [4.5 1.5]
 [1.4 0.2]
 [6.7 2. ]]


#### Sklearn

Train Model 

In [14]:
# Create linear regression object
regr_MV = linear_model.LinearRegression()

# Train the model using the training sets
regr_MV.fit(X_train_MVreg, y_train_MVreg)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [15]:
# The coefficients
print('Sklearn Coefficients: \n', regr_MV.coef_, "\n")
print('Sklearn Intercepts: \n', regr_MV.intercept_)

Sklearn Coefficients: 
 [[ 0.65220929 -3.04039014 -0.86658461]
 [ 0.12109241 -1.5742572  -0.61824543]] 

Sklearn Intercepts: 
 [1.27022853 1.21825327]


Same as fitting linear model 2 seperate times for each y variable.

In [16]:
# petal_length
regr_MV1 = linear_model.LinearRegression()

# Train the model using the training sets
regr_MV1.fit(X_train_MVreg, y_train_MVreg[:, 0])

print('Sklearn Coefficients: \n', regr_MV1.coef_)
print('Sklearn Intercept: \n', regr_MV1.intercept_)

Sklearn Coefficients: 
 [ 0.65220929 -3.04039014 -0.86658461]
Sklearn Intercept: 
 1.270228526905886


In [17]:
# petal_width
regr_MV2 = linear_model.LinearRegression()

# Train the model using the training sets
regr_MV2.fit(X_train_MVreg, y_train_MVreg[:, 1])

print('Sklearn Coefficients: \n', regr_MV2.coef_)
print('Sklearn Intercept: \n', regr_MV2.intercept_)

Sklearn Coefficients: 
 [ 0.12109241 -1.5742572  -0.61824543]
Sklearn Intercept: 
 1.2182532653067053


Test Model

In [18]:
# Make predictions using the testing set
y_pred_MV = regr_MV.predict(X_test_MVreg)
print("Sklearn MSE: ", mean_squared_error(y_test_MVreg, y_pred_MV))

Sklearn MSE:  0.05320301125486809


#### Statsmodels

Using Statsmodels to fit 2 seperate models. Does not allow for multivariate regression. 

In [19]:
X_train_MVreg_SM = sm.add_constant(X_train_MVreg)
# Fit and summarize OLS model
mod_MV1 = sm.OLS(y_train_MVreg[:,0], X_train_MVreg_SM).fit()

In [20]:
print("Statmodels Coefficients:")
short_summary(mod_MV1)

Statmodels Coefficients:


,coef,std err,t,P>|t|,[0.025,0.975]
const,1.2702,0.356,3.572,0.001,0.564,1.976
x1,0.6522,0.054,12.135,0.000,0.546,0.759
x2,-3.0404,0.111,-27.420,0.000,-3.260,-2.820
x3,-0.8666,0.078,-11.070,0.000,-1.022,-0.711


In [21]:
mod_MV2 = sm.OLS(y_train_MVreg[:,1], X_train_MVreg_SM).fit()
print("Statmodels Coefficients:")
short_summary(mod_MV2)

Statmodels Coefficients:


,coef,std err,t,P>|t|,[0.025,0.975]
const,1.2183,0.256,4.766,0.000,0.711,1.726
x1,0.1211,0.039,3.134,0.002,0.044,0.198
x2,-1.5743,0.080,-19.750,0.000,-1.732,-1.416
x3,-0.6182,0.056,-10.986,0.000,-0.730,-0.507


#### R for proper Multivariate Regression 

Using R within python to provide proper context on statistical tests in multivariate regression since sklearn does not provide hypothesis test results for multivariate regression and statsmodel does not have multivariate regression functionality at all.     
   
The _pyper_ library enables this and can be installed using `pip install pyper`. to install

In [22]:
import pyper as pr

In [23]:
# CREATE A R INSTANCE WITH PYPER
r=pr.R(RCMD="C:/Program Files/R/R-4.0.0/bin/R", use_pandas = True)

In [24]:
#convert arrays to dataframes
X_train_MVreg_df=pd.DataFrame(X_train_MVreg, columns=['sepal_length', 'Iris-virginica', 'Iris-versicolor']) 
X_test_MVreg_df=pd.DataFrame(X_test_MVreg, columns=['sepal_length', 'Iris-virginica', 'Iris-versicolor']) 
y_train_MVreg_df=pd.DataFrame(y_train_MVreg, columns=['petal_length', 'petal_width']) 
y_test_MVreg_df=pd.DataFrame(y_test_MVreg, columns=['petal_length', 'petal_width']) 

In [25]:
X_train_MVreg_df.head()

,sepal_length,Iris-virginica,Iris-versicolor
0,5.7,0.0,1.0
1,7.6,0.0,0.0
2,5.6,0.0,1.0
3,5.1,1.0,0.0
4,7.7,0.0,0.0


In [26]:
y_train_MVreg_df.head()

,petal_length,petal_width
0,4.2,1.3
1,6.6,2.1
2,4.5,1.5
3,1.4,0.2
4,6.7,2.0


In [27]:
# PASS DATA FROM PYTHON TO R
r.assign("X_Train", X_train_MVreg_df)
r.assign("Y_Train", y_train_MVreg_df) 
r.assign("X_Test", X_test_MVreg_df)
r.assign("Y_Test", y_test_MVreg_df) 

In [28]:
r("Y_Train <- as.matrix(Y_Train)")
r("mvmod <- lm(Y_Train ~ sepal_length + Iris.virginica + Iris.versicolor, data=X_Train)")
print(r("summary(mvmod)"))

try({summary(mvmod)})
Response petal_length :

Call:
lm(formula = petal_length ~ sepal_length + Iris.virginica + Iris.versicolor, 
    data = X_Train)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.81265 -0.16505  0.00125  0.14392  0.78310 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)      1.27023    0.35557   3.572 0.000555 ***
sepal_length     0.65221    0.05375  12.135  < 2e-16 ***
Iris.virginica  -3.04039    0.11088 -27.420  < 2e-16 ***
Iris.versicolor -0.86658    0.07828 -11.070  < 2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 0.2849 on 96 degrees of freedom
Multiple R-squared:  0.9738,	Adjusted R-squared:  0.973 
F-statistic:  1191 on 3 and 96 DF,  p-value: < 2.2e-16


Response petal_width :

Call:
lm(formula = petal_width ~ sepal_length + Iris.virginica + Iris.versicolor, 
    data = X_Train)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.55692 -0.11040 -0.

In [29]:
print(r("mvmod"))

try({mvmod})

Call:
lm(formula = Y_Train ~ sepal_length + Iris.virginica + Iris.versicolor, 
    data = X_Train)

Coefficients:
                 petal_length  petal_width
(Intercept)       1.2702        1.2183    
sepal_length      0.6522        0.1211    
Iris.virginica   -3.0404       -1.5743    
Iris.versicolor  -0.8666       -0.6182    




In [30]:
print(r("library(car)"))
print(r("Anova(mvmod)"))

try({library(car)})
Loading required package: carData

try({Anova(mvmod)})

Type II MANOVA Tests: Pillai test statistic
                Df test stat approx F num Df den Df    Pr(>F)    
sepal_length     1   0.61101    74.61      2     95 < 2.2e-16 ***
Iris.virginica   1   0.89783   417.40      2     95 < 2.2e-16 ***
Iris.versicolor  1   0.64406    85.95      2     95 < 2.2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1



In [31]:
#r("Y_Test <- as.matrix(Y_Test)")
r("ypred=predict(mvmod, X_Test) ")

'try({ypred=predict(mvmod, X_Test) })\r\n'

In [35]:
print(r('ypred'))

try({ypred})
   petal_length petal_width
0      4.382121   1.3386715
1      1.947431   0.3342228
2      6.292240   2.1506648
3      4.316900   1.3265623
4      4.838667   1.4234362
5      1.751769   0.2978951
6      4.056016   1.2781253
7      5.770473   2.0537909
8      4.447342   1.3507808
9      4.186458   1.3023438
10     5.509589   2.0053539
11     1.360443   0.2252396
12     1.816989   0.3100043
13     1.425664   0.2373489
14     1.556106   0.2615674
15     4.512562   1.3628900
16     5.509589   2.0053539
17     4.056016   1.2781253
18     4.121237   1.2902346
19     5.444368   1.9932447
20     1.295222   0.2131304
21     5.248705   1.9569170
22     1.490885   0.2494581
23     5.444368   1.9932447
24     6.422682   2.1748833
25     5.640031   2.0295724
26     5.640031   2.0295724
27     5.705252   2.0416817
28     1.360443   0.2252396
29     1.360443   0.2252396
30     1.230001   0.2010212
31     1.947431   0.3342228
32     4.773446   1.4113270
33     1.360443   0.2252396
34     

In [37]:
r("mean((Y_Test - ypred)^2)")


'try({mean((Y_Test - ypred)^2)})\r\n[1] NA\r\nWarning message:\r\nIn mean.default((Y_Test - ypred)^2) :\r\n  argument is not numeric or logical: returning NA\r\n'